<a href="https://colab.research.google.com/github/SkyDriver1979/torque-calculation/blob/main/Torque_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Setup
!rm -rf /content/torque-calculation
!git clone https://github.com/SkyDriver1979/torque-calculation.git
#!sudo apt install tesseract-ocr libtesseract-dev
!sudo add-apt-repository -y ppa:alex-p/tesseract-ocr5
!sudo apt update
!sudo apt install tesseract-ocr
!pip -q install pytesseract

import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
from operator import itemgetter
%matplotlib inline

diagram = cv2.imread("torque-calculation/diagram.jpg")
plt.imshow(diagram[:,:,::-1])

In [2]:
# @title Processing Functions

def extract_text(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    denoised_image = cv2.fastNlMeansDenoising(gray_image, None, 10, 7, 21)
    return pytesseract.image_to_string(denoised_image)

def extract_diagram_image(image, diagram):
    match_result = cv2.matchTemplate(image, diagram, cv2.TM_CCOEFF)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(match_result)
    h = max_loc[1]
    w = max_loc[0]
    matched_diagram = image[h:h+diagram.shape[0],w:w+diagram.shape[1],:]
    return matched_diagram

def extract_numbers(image):
    def extract_digits(text):
        return ''.join(c for c in text if c.isdigit())

    text = extract_text(image).split("\n")
    text = [line for line in text if len(extract_digits(line))>0]
    bph_max = int(extract_digits(text[0]))
    kgfm_max = int(extract_digits(text[1]))
    rpm_text = [line for line in text if "rpm" in line][0]
    rpm_split = rpm_text.split("rpm")
    rpm_min = int(extract_digits(rpm_split[0]))
    rpm_max = int(extract_digits(rpm_split[1]))
    return rpm_min, rpm_max, kgfm_max, bph_max

def extract_car_name(image):
    ratio = image.shape[0]/2160
    h_min = int(420 * ratio)
    h_max = int(500 * ratio)
    w_max = int(900 * ratio)
    return extract_text(image[h_min:h_max,:w_max,:]).strip()


def find_coordinates(gray_diagram, colour_diagram):
    def is_cyan(pixel):
        return pixel[0] / pixel[2] >= 1.3 and pixel[1] / pixel[2] >= 1.3

    def is_bhp_region(region, x, ymin, colour_diagram):
        for y in range(region[0],region[1]+1):
            if is_cyan(colour_diagram[y+ymin,x]):
                return True
        return False

    def find_regions(slice):
        #print(slice.shape)
        result = []
        in_region = False
        region_start = 0
        for i in range(len(slice)):
            if in_region:
                if not slice[i]:
                    in_region = False
                    result.append((region_start,i-1))
            else:
                if slice[i]:
                    region_start = i
                    in_region = True
        if in_region:
            result.append((region_start,len(slice)-1))
        return result

    kgfm = []
    bhp = []
    working_diagram = gray_diagram > 100

    xaxis = np.count_nonzero(gray_matched_diagram > 60,axis=0) > gray_matched_diagram.shape[0] * 0.5
    #find where X axis begin in image
    for i in range(len(xaxis)//2):
        if xaxis[i]:
            xmin = i
            break

    #find where X axis ends in the image
    for i in range(len(xaxis)//2,len(xaxis)):
        if xaxis[i]:
            xmax = i

    #find where is Y start
    yaxis = np.count_nonzero(working_diagram,axis=1) > gray_matched_diagram.shape[1] * 0.5
    for i in range(len(yaxis)):
        if yaxis[i]:
            ymax = i
            break

    #find where diagram stgart on Y axis
    yaxis = np.count_nonzero(gray_matched_diagram < 45,axis=1) > gray_matched_diagram.shape[1] * 0.5
    for i in range(len(yaxis)):
        if yaxis[i]:
            ymin = i
            break
    #print(xmin,xmax,ymin,ymax)

    for x in range(xmin, xmax+1):
        found_kgfm = None
        found_bhp = None
        regions = find_regions(working_diagram[ymin:ymax,x])
        if len(regions) >= 3:
            regions = regions[-2:]
        if len(regions) == 2:
            for region in regions:
                if is_bhp_region(region, x, ymin, colour_diagram):
                    found_bhp = region
                else:
                    found_kgfm = region
        else:
            found_kgfm = region

        if found_bhp is not None:
            bhp.append((x,ymax - found_bhp[0]))
        if found_kgfm is not None:
            kgfm.append((x, ymax - found_kgfm[0]))
    return kgfm , bhp

def normalize_values(values, rpm_min, rpm_max, y_out_max):
    def linear_function(x1, y1, x2, y2):
        a = (y2-y1)/(x2-x1)
        b = y1 - a * x1
        return a, b
    y_in_max = max(values, key=itemgetter(1))[1]
    rpm_in_min = values[0][0]
    rpm_in_max = values[-1][0]
    rpm_a, rpm_b = linear_function(rpm_in_min,rpm_min, rpm_in_max, rpm_max)
    y_a, y_b = linear_function(0, 0, y_in_max, y_out_max)

    return [(rpm_a * x + rpm_b, y_a * y + y_b) for x, y in values]

In [ ]:
# @title Input parameters
screenshot = 'torque-calculation/sample-screenshot.jpg' # @param {type:"string"}
gear_1 = 2.923 # @param {type:"number"}
gear_2 = 2.267 # @param {type:"number"}
gear_3 = 1.75 # @param {type:"number"}
gear_4 = 1.444 # @param {type:"number"}
gear_5 = 1.19 # @param {type:"number"}
gear_6 = 1 # @param {type:"number"}
gear_7 = 0 # @param {type:"number"}
final_ratio = 2.95 # @param {type:"number"}
tyre_diameter_in_meters = 0.698 # @param {type:"number"}

img = cv2.imread(screenshot,cv2.IMREAD_COLOR)
assert img is not None, "Screenshot filename is not correct"
print(extract_car_name(img))
plt.imshow(img[:,:,::-1])

In [ ]:
# @title Finding a diagram and extracting values from diagram
matched_diagram = extract_diagram_image(img, diagram)
gray_matched_diagram = cv2.cvtColor(matched_diagram, cv2.COLOR_BGR2GRAY)
kgfm, bhp = find_coordinates(gray_matched_diagram,matched_diagram)
rpm_min, rpm_max, kgfm_max, bhp_max = extract_numbers(matched_diagram)
print(f"rpm_min = {rpm_min}")
print(f"rpm_max = {rpm_max}")
print(f"kgfm_max = {kgfm_max}")
print(f"bhp_max = {bhp_max}")
normalized_kgfm = normalize_values(kgfm, rpm_min, rpm_max, kgfm_max)
normalized_bhp = normalize_values(bhp, rpm_min, rpm_max, bhp_max)
plt.imshow(matched_diagram[:,:,::-1])

In [ ]:
print(extract_numbers(diagram))
print(extract_car_name(img))
print(max(normalized_kgfm, key=itemgetter(1)))
print(max(normalized_bhp, key=itemgetter(1)))
